In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
!pip install torchaudio
!pip install PyDrive
!pip install soundfile

     |████████████████████████████████| 3.2MB 5.0MB/s 


In [4]:
! git clone https://github.com/facebookresearch/CPC_audio.git

Cloning into 'CPC_audio'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 84 (delta 13), reused 75 (delta 6), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [5]:
%cd /content/CPC_audio
!python setup.py develop

/content/CPC_audio
Compiling cpc/eval/ABX/dtw.pyx because it changed.
[1/1] Cythonizing cpc/eval/ABX/dtw.pyx
running develop
running egg_info
creating CPC_audio.egg-info
writing CPC_audio.egg-info/PKG-INFO
writing dependency_links to CPC_audio.egg-info/dependency_links.txt
writing top-level names to CPC_audio.egg-info/top_level.txt
writing manifest file 'CPC_audio.egg-info/SOURCES.txt'
writing manifest file 'CPC_audio.egg-info/SOURCES.txt'
running build_ext
building 'cpc.eval.ABX.dtw' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/cpc
creating build/temp.linux-x86_64-3.6/cpc/eval
creating build/temp.linux-x86_64-3.6/cpc/eval/ABX
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I/usr/include/python3.6m -c cpc/eval/ABX/dtw.c -o build/temp.linux-x86_64-3.6/cpc/eval/ABX/

In [6]:
import torch
import torchaudio

# Part 1 : contrastive predictive coding

Contrastive Predictive Coding (CPC) is a method of unsupervised training for speech models. The idea behind it is pretty simple:


1.   The raw audio wave is passed through a convolutional network: the ```encoder```
2.   Then, the encoder's output is given to a recurrent network the ```context```
3. A third party network, the ```prediction_network``` will try to predict the  future embeddings of the encoder using the output of the context network.

In order to avoid a collapse to trivial solutions, the prediction_network doesn't try to reconstruct the future features. Instead, using the context output $c_t$ at time $t$ it is trained to discriminate the real  encoder representatioin $g_{t+k}$ at time $t+k$ from several other features $(g_n)_n$ taken elsewhere in the batch. Thus the loss becomes:

\\[ \mathcal{L}_c = - \frac{1}{K} \sum_{k=1}^K \text{Cross_entropy}(\phi_k(c_t), g_{t+k}) \\]

Or:

\\[ \mathcal{L}_c = - \frac{1}{K} \sum_{k=1}^K \log \frac{ \exp\left(\phi_k(c_t)^\top g_{t+k}\right) }{  \sum_{\mathbf{n}\in\mathcal{N}_t} \exp\left(\phi_k(c_t)^\top g_n\right)} \\]

Where:


*   $\phi_k$ is the prediction network for the kth timestep
*   $\mathcal{N}_t$ is the set of all negative examples sampled for timestep $t$




## Exercice 1 : Building the model

In this exercise, we will build and train a small CPC model using the repository CPC_audio.

The code below loads a context and an encoder newtorks.

In [25]:
%cd /content/CPC_audio
from cpc.model import CPCEncoder, CPCAR
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

DIM_ENCODER=256
DIM_CONTEXT=256
KEEP_HIDDEN_VECTOR=False
N_LEVELS_CONTEXT=1
CONTEXT_RNN="LSTM"
N_PREDICTIONS=12
LEARNING_RATE=2e-4
N_NEGATIVE_SAMPLE =128

/content/CPC_audio


In [26]:
encoder = CPCEncoder(DIM_ENCODER).to(device)
context = CPCAR(DIM_ENCODER, DIM_CONTEXT, KEEP_HIDDEN_VECTOR, 1, mode=CONTEXT_RNN).to(device)

In [27]:
# Several functions that will be necessary to load the data later
from cpc.dataset import findAllSeqs, AudioBatchData, parseSeqLabels
SIZE_WINDOW = 20480
BATCH_SIZE=8
def load_dataset(path_dataset, file_extension='.wav', phone_label_dict=None):
  data_list, speakers = findAllSeqs(path_dataset, extension=file_extension)
  dataset = AudioBatchData(path_dataset, SIZE_WINDOW, data_list, phone_label_dict, len(speakers))
  return dataset

Now build a new class, ```CPCModel``` which will

In [28]:
class CPCModel(torch.nn.Module):

    def __init__(self,
                 encoder,
                 AR):

        super(CPCModel, self).__init__()
        self.gEncoder = encoder
        self.gAR = AR

    def forward(self, batch_data):
        

        encoder_output = self.gEncoder(batch_data)
        #print(encoder_output.shape)
        # The output of the encoder data does not have the good format 
        # indeed it is Batch_size x Hidden_size x temp size
        # while the context requires Batch_size  x temp size x Hidden_size
        # thus you need to permute
        context_input = encoder_output.permute(0, 2, 1)

        context_output = self.gAR(context_input)
        #print(context_output.shape)
        return context_output, encoder_output

In [29]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Let's test your code !


In [30]:
audio = torchaudio.load("/content/drive/My Drive/speech/data/train/200702-000842_wol_4c3_elicit/200702-000842_wol_4c3_elicit_0.wav")[0]
audio = audio.view(1, 1, -1)
cpc_model = CPCModel(encoder, context).to(device)
context_output, encoder_output = cpc_model(audio.to(device))

## Exercise 2 : CPC loss

We will define a class ```CPCCriterion``` which will hold the prediction networks $\phi_k$ defined above and perform the classification loss $\mathcal{L}_c$.

a) In this exercise, the $\phi_k$ will be a linear transform, ie:

\\[ \phi_k(c_t) = \mathbf{A}_k c_t\\]

Using the class [torch.nn.Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear), define the transformations $\phi_k$ in the code below and complete the function ```get_prediction_k``` which computes $\phi_k(c_t)$ for a given batch of vectors $c_t$.

b) Using both ```get_prediction_k```  and ```sample_negatives``` defined below, write the forward function which will take as input two batches of features $c_t$ and $g_t$ and outputs the classification loss $\mathcal{L}_c$ and the average acuracy for all predictions. 

In [31]:
# Exercice 2: write the CPC loss
# a) Write the negative sampling (with some help)
# ERRATUM: it's really hard, the sampling will be provided

class CPCCriterion(torch.nn.Module):

  def __init__(self,
               K,
               dim_context,
               dim_encoder,
               n_negative):
    super(CPCCriterion, self).__init__()
    self.K_ = K
    self.dim_context = dim_context
    self.dim_encoder = dim_encoder
    self.n_negative = n_negative

    self.predictors = torch.nn.ModuleList() 
    for k in range(self.K_):
      # TO COMPLETE !
      
      # A affine transformation in pytorch is equivalent to a nn.Linear layer
      # To get a linear transformation you must set bias=False
      # input dimension of the layer = dimension of the encoder
      # output dimension of the layer = dimension of the context
      self.predictors.append(torch.nn.Linear(dim_context, dim_encoder, bias=False))

  def get_prediction_k(self, context_data):

    #TO COMPLETE !
    output = [] 
    # For each time step k
    for k in range(self.K_):

      # We need to compute phi_k = A_k * c_t
      phi_k = self.predictors[k](context_data)
      output.append(phi_k)

    return output


  def sample_negatives(self, encoded_data):
    r"""
    Sample some negative examples in the given encoded data.
    Input:
    - encoded_data size: B x T x H
    Returns
    - outputs of size B x (n_negative + 1) x (T - K_) x H
      outputs[:, 0, :, :] contains the positive example
      outputs[:, 1:, :, :] contains negative example sampled in the batch
    - labels, long tensor of size B x (T - K_)
      Since the positive example is always at coordinates 0 for all sequences 
      in the batch and all timestep in the sequence, labels is just a tensor
      full of zeros !
    """
    batch_size, time_size, dim_encoded = encoded_data.size()
    window_size = time_size - self.K_
    outputs = []

    neg_ext = encoded_data.contiguous().view(-1, dim_encoded)
    n_elem_sampled = self.n_negative * window_size * batch_size
    # Draw nNegativeExt * batchSize negative samples anywhere in the batch
    batch_idx = torch.randint(low=0, high=batch_size,
                              size=(n_elem_sampled, ),
                              device=encoded_data.device)

    seq_idx = torch.randint(low=1, high=time_size,
                            size=(n_elem_sampled, ),
                            device=encoded_data.device)

    base_idx = torch.arange(0, window_size, device=encoded_data.device)
    base_idx = base_idx.view(1, 1, window_size)
    base_idx = base_idx.expand(1, self.n_negative, window_size)
    base_idx = base_idx.expand(batch_size, self.n_negative, window_size)
    seq_idx += base_idx.contiguous().view(-1)
    seq_idx = torch.remainder(seq_idx, time_size)

    ext_idx = seq_idx + batch_idx * time_size
    neg_ext = neg_ext[ext_idx].view(batch_size, self.n_negative,
                                    window_size, dim_encoded)
    label_loss = torch.zeros((batch_size, window_size),
                              dtype=torch.long,
                              device=encoded_data.device)

    for k in range(1, self.K_ + 1):

      # Positive samples
      if k < self.K_:
          pos_seq = encoded_data[:, k:-(self.K_-k)]
      else:
          pos_seq = encoded_data[:, k:]

      pos_seq = pos_seq.view(batch_size, 1, pos_seq.size(1), dim_encoded)
      full_seq = torch.cat((pos_seq, neg_ext), dim=1)
      outputs.append(full_seq)

    return outputs, label_loss

  def forward(self, encoded_data, context_data):

    # TO COMPLETE:
    # Perform the full cpc criterion
    # Returns 2 values:
    # - the average classification loss avg_loss
    # - the average classification acuracy avg_acc

    # Reminder : The permuation !
    encoded_data = encoded_data.permute(0, 2, 1)

    # First we need to sample the negative examples
    negative_samples, labels = self.sample_negatives(encoded_data)

    # Then we must compute phi_k
    phi_k = self.get_prediction_k(context_data)

    # Finally we must get the dot product between phi_k and negative_samples 
    # for each k

    #The total loss is the average of all losses
    avg_loss = 0

    # Average acuracy
    avg_acc = 0

    for k in range(self.K_):
      B, N_sampled, S_small, H = negative_samples[k].size() 
      B, S, H = phi_k[k].size()

      # As told before S = S_small + K. For segments too far in the sequence
      # there are no positive exmples anyway, so we must shorten phi_k
      phi = phi_k[k][:, :S_small]

      # Now the dot product
      # You have several ways to do that, let's do the simple but non optimal 
      # one
      # pytorch has a matrix product function https://pytorch.org/docs/stable/torch.html#torch.bmm
      # But it takes only 3D tensors of the same batch size !
      # To begin negative_samples is a 4D tensor ! 
      # We want to compute the dot product for each features, of each sequence
      # of the batch. Thus we are trying to compute a dot product for all
      # B* N_sampled * S_small 1D vector of negative_samples[k]
      # Or, a 1D tensor of size H is also a matrix of size 1 x H
      # Then, we must view it as a 3D tensor of size (B* N_sampled * S_small, 1, H)
      negative_sample_k  =  negative_samples[k].view(B* N_sampled* S_small, 1, H)

      # But now phi and negative_sample_k no longer have the same batch size !
      # No worries, we can expand phi so that each sequence of the batch
      # is repeated N_sampled times
      phi = phi.view(B, 1,S_small, H).expand(B, N_sampled, S_small, H)

      # And now we can view it as a 3D tensor 
      phi  = phi.contiguous().view(B * N_sampled * S_small, H, 1)

      # We can finally get the dot product !
      scores = torch.bmm(negative_sample_k, phi)

      # Dot_product has a size (B * N_sampled * S_small , 1, 1)
      # Let's reorder it a bit
      scores = scores.reshape(B, N_sampled, S_small)

      # For each elements of the sequence, and each elements sampled, it gives 
      # a floating score stating the likelihood of this element being the 
      # true one.
      # Now the classification loss, we need to use the Cross Entropy loss
      # https://pytorch.org/docs/master/generated/torch.nn.CrossEntropyLoss.html

      # For each time-step of each sequence of the batch 
      # we have N_sampled possible predictions. 
      # Looking at the documentation of torch.nn.CrossEntropyLoss
      # we can see that this loss expect a tensor of size M x C where 
      # - M is the number of elements with a classification score
      # - C is the number of possible classes
      # There are N_sampled candidates for each predictions so
      # C = N_sampled 
      # Each timestep of each sequence of the batch has a prediction so
      # M = B * S_small
      # Thus we need an input vector of size B * S_small, N_sampled
      # To begin, we need to permute the axis
      scores = scores.permute(0, 2, 1) # Now it has size B , S_small, N_sampled

      # Then we can cast it into a 2D tensor
      scores = scores.reshape(B * S_small, N_sampled)

      # Same thing for the labels 
      labels = labels.reshape(B * S_small)

      # Finally we can get the classification loss
      loss_criterion = torch.nn.CrossEntropyLoss()
      loss_k = loss_criterion(scores, labels)
      avg_loss+= loss_k

      # And for the acuracy
      # The prediction for each elements is the sample with the highest score
      # Thus the tensors of all predictions is the tensors of the index of the 
      # maximal score for each time-step of each sequence of the batch
      predictions = torch.argmax(scores, 1)
      acc_k  = (labels == predictions).sum() / (B * S_small)
      avg_acc += acc_k

    # Normalization
    avg_loss = avg_loss / self.K_
    avg_acc = avg_acc / self.K_
      
    return avg_loss , avg_acc

Don't forget to test !

In [32]:
audio = torchaudio.load("/content/drive/My Drive/speech/data/train/200702-000842_wol_4c3_elicit/200702-000842_wol_4c3_elicit_0.wav")[0]
audio = audio.view(1, 1, -1)
cpc_criterion = CPCCriterion(N_PREDICTIONS, DIM_CONTEXT, 
                             DIM_ENCODER, N_NEGATIVE_SAMPLE).to(device)
context_output, encoder_output = cpc_model(audio.to(device))
loss, avg = cpc_criterion(encoder_output,context_output)

## Exercise 3: Full training loop !

You have the model, you have the criterion. All you need now are a data loader and an optimizer to run your training loop.

We will use an Adam optimizer:

In [33]:
parameters = list(cpc_criterion.parameters()) + list(cpc_model.parameters())
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)

And as far as the data loader is concerned, we will rely on the data loader provided by the CPC_audio library.

In [34]:
dataset_train = load_dataset('/content/drive/My Drive/speech/data/train')
dataset_val = load_dataset('/content/drive/My Drive/speech/data/validation')
data_loader_train = dataset_train.getDataLoader(BATCH_SIZE, "speaker", True)
data_loader_val = dataset_train.getDataLoader(BATCH_SIZE, "sequence", False)

6it [00:00, 470.97it/s]

Saved cache file at /content/drive/My Drive/speech/data/train/_seqs_cache.txt



0it [00:00, ?it/s]

Checking length...


242it [00:00, 118952.49it/s]


...

4it [00:00, 68.29it/s]

Joined process, elapsed=0.928 secs
Saved cache file at /content/drive/My Drive/speech/data/validation/_seqs_cache.txt



147it [00:00, 40005.37it/s]

Checking length...
Done, elapsed: 0.146 seconds
Scanned 147 sequences in 0.15 seconds
1 chunks computed
Joining pool


Joined process, elapsed=1.358 secs


Now that everything is ready, complete and test the ```train_step``` function below which trains the model for one epoch.

In [35]:
def train_step(data_loader,
               cpc_model,
               cpc_criterion,
               optimizer):
  
  avg_loss = 0
  avg_acc = 0
  n_items = 0

  for step, data in enumerate(data_loader):
    x,y = data
    bs = len(x)
    optimizer.zero_grad()
    context_output, encoder_output = cpc_model(x.to(device))
    loss , acc = cpc_criterion(encoder_output, context_output)
    loss.backward()
    n_items+=bs
    avg_loss+=loss.item()*bs
    avg_acc +=acc.item()*bs
  
  avg_loss/=n_items
  avg_acc/=n_items
  return avg_loss, avg_acc

## Exercise 4 : Validation loop

Now complete the validation loop.

In [36]:
def validation_step(data_loader,
                    cpc_model,
                    cpc_criterion):
  
  avg_loss = 0
  avg_acc = 0
  n_items = 0

  for step, data in enumerate(data_loader):
    x,y = data
    bs = len(x)
    context_output, encoder_output = cpc_model(x.to(device))
    loss , acc = cpc_criterion(encoder_output, context_output)
    n_items+=bs
    avg_loss+=loss.item()*bs
    avg_acc+=acc.item()*bs
  
  avg_loss/=n_items
  avg_acc/=n_items
  return avg_loss, avg_acc

## Exercise 5: Run everything

In [37]:
def run(train_loader,
        val_loader,
        cpc_model,
        cpc_criterion,
        optimizer,
        n_epochs):
  
  for epoch in range(n_epochs):

    
    print(f"Running epoch {epoch+1} / {n_epochs}")
    avg_loss_train, avg_acc_train = train_step(train_loader, cpc_model, cpc_criterion, optimizer)
    print("----------------------")
    print(f"Training dataset")
    print(f"- average loss : {avg_loss_train}")
    print(f"- average acuracy : {avg_acc_train}")
    print("----------------------")
    with torch.no_grad():
      cpc_model.eval()
      cpc_criterion.eval()
      avg_loss_val, avg_acc_val = validation_step(val_loader, cpc_model, cpc_criterion)
      print(f"Validation dataset")
      print(f"- average loss : {avg_loss_val}")
      print(f"- average acuracy : {avg_acc_val}")
      print("----------------------")
      print()
      cpc_model.train()
      cpc_criterion.train()

In [38]:
run(data_loader_train, data_loader_val, cpc_model,cpc_criterion,optimizer,1)

Running epoch 1 / 1
----------------------
Training dataset
- average loss : 4.860483307262947
- average acuracy : 0.0
----------------------
Validation dataset
- average loss : 4.86048201856942
- average acuracy : 0.0
----------------------



Once everything is donw, clear the memory.

In [39]:
del dataset_train
del dataset_val
del cpc_model
del context
del encoder

# Part 2 : Fine tuning

## Exercice 1 : Phone separability with aligned phonemes.

One option to evaluate the quality of the features trained with CPC can be to check if they can be used to recognize phonemes. 
To do so, we can fine-tune a pre-trained model using a limited amount of labelled speech data.
We are going to start with a simple evaluation setting where we have the phone labels for each timestep corresponding to a CPC feature.

We will work with a model already pre-trained on English data. As far as the fine-tuning dataset is concerned, we will use a 1h subset of [librispeech-100](http://www.openslr.org/12/). 

In [40]:
!mkdir checkpoint_data
!wget https://dl.fbaipublicfiles.com/librilight/CPC_checkpoints/not_hub/2levels_6k_top_ctc/checkpoint_30.pt -P checkpoint_data
!wget https://dl.fbaipublicfiles.com/librilight/CPC_checkpoints/not_hub/2levels_6k_top_ctc/checkpoint_logs.json -P checkpoint_data
!wget https://dl.fbaipublicfiles.com/librilight/CPC_checkpoints/not_hub/2levels_6k_top_ctc/checkpoint_args.json -P checkpoint_data
!ls checkpoint_data

mkdir: cannot create directory ‘checkpoint_data’: File exists
--2020-07-05 23:16:53--  https://dl.fbaipublicfiles.com/librilight/CPC_checkpoints/not_hub/2levels_6k_top_ctc/checkpoint_30.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113599715 (108M) [application/octet-stream]
Saving to: ‘checkpoint_data/checkpoint_30.pt.1’

checkpoint_30.pt.1  100%[===================>] 108.34M  93.1MB/s    in 1.2s    

2020-07-05 23:16:54 (93.1 MB/s) - ‘checkpoint_data/checkpoint_30.pt.1’ saved [113599715/113599715]

--2020-07-05 23:16:56--  https://dl.fbaipublicfiles.com/librilight/CPC_checkpoints/not_hub/2levels_6k_top_ctc/checkpoint_logs.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbai

In [44]:
%cd /content/CPC_audio
from cpc.dataset import parseSeqLabels
from cpc.feature_loader import loadModel

checkpoint_path = 'checkpoint_data/checkpoint_30.pt'
cpc_model, HIDDEN_CONTEXT_MODEL, HIDDEN_ENCODER_MODEL = loadModel([checkpoint_path])
cpc_model = cpc_model.cuda()
label_dict, N_PHONES = parseSeqLabels('/content/drive/My Drive/speech/all_sessions.txt')
dataset_train = load_dataset('/content/drive/My Drive/speech/data/train', file_extension='.wav', phone_label_dict=label_dict)
dataset_val = load_dataset('/content/drive/My Drive/speech/data/validation', file_extension='.wav', phone_label_dict=label_dict)
data_loader_train = dataset_train.getDataLoader(BATCH_SIZE, "speaker", True)
data_loader_val = dataset_val.getDataLoader(BATCH_SIZE, "sequence", False)

/content/CPC_audio
Loading checkpoint checkpoint_data/checkpoint_30.pt
Loading the state dict at checkpoint_data/checkpoint_30.pt


6it [00:00, 551.28it/s]

Saved cache file at /content/drive/My Drive/speech/data/train/_seqs_cache.txt


Checking length...


242it [00:00, 104469.08it/s]


Done, elapsed: 0.261 seconds
Scanned 242 sequences in 0.26 seconds
1 chunks computed
Joining pool
Joined process, elapsed=1.067 secs


KeyError: ignored

In [ ]:
??cpc_model

Then we will use a simple linear classifier to recognize the phonemes from the features produced by ```cpc_model```. 

### a) Build the phone classifier 

Design a class of linear classifiers, ```PhoneClassifier``` that will take as input a batch of sequences of CPC features and output a score vector for each phoneme

In [ ]:
class PhoneClassifier(torch.nn.Module):

  def __init__(self,
               input_dim : int,
               n_phones : int):
    super(PhoneClassifier, self).__init__()
    self.linear = torch.nn.Linear(input_dim, n_phones)
    

  def forward(self, x):
    return self.linear(x)

Our phone classifier will then be:

In [43]:
phone_classifier = PhoneClassifier(HIDDEN_CONTEXT_MODEL, N_PHONES).to(device)

NameError: ignored

### b - What would be the correct loss criterion for this task ?



In [ ]:
loss_criterion = torch.nn.CrossEntropyLoss()

To perform the fine-tuning, we will also need an optimization function.

We will use an [Adam optimizer ](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam).

In [ ]:
parameters = list(phone_classifier.parameters()) + list(cpc_model.parameters())
LEARNING_RATE = 2e-4
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)

You might also want to perform this training while freezing the weights of the ```cpc_model```. Indeed, if the pre-training was good enough, then ```cpc_model``` phonemes representation should be linearly separable. In this case the optimizer should be defined like this:

In [ ]:
optimizer_frozen = torch.optim.Adam(list(phone_classifier.parameters()), lr=LEARNING_RATE)

### c- Now let's build a training loop. 
Complete the function ```train_one_epoch``` below.



In [ ]:
def train_one_epoch(cpc_model, 
                    phone_classifier, 
                    loss_criterion, 
                    data_loader, 
                    optimizer):

  cpc_model.train()
  loss_criterion.train()

  avg_loss = 0
  avg_accuracy = 0
  n_items = 0
  for step, full_data in enumerate(data_loader):
    # Each batch is represented by a Tuple of vectors:
    # sequence of size : N x 1 x T
    # label of size : N x T
    # 
    # With :
    # - N number of sequence in the batch
    # - T size of each sequence
    sequence, label = full_data
    
    

    bs = len(sequence)
    seq_len = label.size(1)
    optimizer.zero_grad()
    context_out, enc_out, _ = cpc_model(sequence.to(device),label.to(device))

    scores = phone_classifier(context_out)

    scores = scores.permute(0,2,1)
    loss = loss_criterion(scores,label.to(device))
    loss.backward()
    optimizer.step()
    avg_loss+=loss.item()*bs
    n_items+=bs
    correct_labels = scores.argmax(1)
    avg_accuracy += ((label==correct_labels.cpu()).float()).mean(1).sum().item()
  avg_loss/=n_items
  avg_accuracy/=n_items
  return avg_loss, avg_accuracy
    

Don't forget to test it !

In [ ]:
avg_loss, avg_accuracy = train_one_epoch(cpc_model, phone_classifier, loss_criterion, data_loader_train, optimizer_frozen)

In [ ]:
avg_loss, avg_accuracy

### d- Build the validation loop

In [ ]:
def validation_step(cpc_model, 
                    phone_classifier, 
                    loss_criterion, 
                    data_loader):
  
  cpc_model.eval()
  phone_classifier.eval()

  avg_loss = 0
  avg_accuracy = 0
  n_items = 0
  with torch.no_grad():
    for step, full_data in enumerate(data_loader):
      # Each batch is represented by a Tuple of vectors:
      # sequence of size : N x 1 x T
      # label of size : N x T
      # 
      # With :
      # - N number of sequence in the batch
      # - T size of each sequence
      sequence, label = full_data
      bs = len(sequence)
      seq_len = label.size(1)
      context_out, enc_out, _ = cpc_model(sequence.to(device),label.to(device))
      scores = phone_classifier(context_out)
      scores = scores.permute(0,2,1)
      loss = loss_criterion(scores,label.to(device))
      avg_loss+=loss.item()*bs
      n_items+=bs
      correct_labels = scores.argmax(1)
      avg_accuracy += ((label==correct_labels.cpu()).float()).mean(1).sum().item()
  avg_loss/=n_items
  avg_accuracy/=n_items
  return avg_loss, avg_accuracy

### e- Run everything

Test this functiion with both ```optimizer``` and ```optimizer_frozen```.

In [ ]:
def run(cpc_model, 
        phone_classifier, 
        loss_criterion, 
        data_loader_train, 
        data_loader_val, 
        optimizer,
        n_epoch):

  for epoch in range(n_epoch):

    print(f"Running epoch {epoch + 1} / {n_epoch}")
    loss_train, acc_train = train_one_epoch(cpc_model, phone_classifier, loss_criterion, data_loader_train, optimizer)
    print("-------------------")
    print(f"Training dataset :")
    print(f"Average loss : {loss_train}. Average accuracy {acc_train}")

    print("-------------------")
    print("Validation dataset")
    loss_val, acc_val = validation_step(cpc_model, phone_classifier, loss_criterion, data_loader_val)
    print(f"Average loss : {loss_val}. Average accuracy {acc_val}")
    print("-------------------")
    print()

In [42]:
run(cpc_model,phone_classifier,loss_criterion,data_loader_train,data_loader_val,optimizer_frozen,n_epoch=10)

NameError: ignored

## Exercise 2 : Phone separability without alignment (PER)

Aligned data are very practical, but un real life they are rarely available. That's why in this excercise we will consider a fine-tuning with non-aligned phonemes.

The model, the optimizer and the phone classifier will stay the same. However, we will replace our phone criterion with a [CTC loss](https://pytorch.org/docs/master/generated/torch.nn.CTCLoss.html). 

In [48]:
loss_ctc = torch.nn.CTCLoss()

Besides, we will use a siglthy different dataset class.

In [50]:
%cd /content/CPC_audio
from cpc.eval.common_voices_eval import SingleSequenceDataset, parseSeqLabels, findAllSeqs

path_train_data_per = '/content/drive/My Drive/speech/data/train'
path_val_data_per = '/content/drive/My Drive/speech/data/validation'
path_test_data_per = '/content/drive/My Drive/speech/data/test'
path_phone_data_per = '/content/drive/My Drive/speech/all_sessions.txt'
BATCH_SIZE=8

phone_labels, N_PHONES = parseSeqLabels(path_phone_data_per)
data_train_per, _ = findAllSeqs(path_train_data_per, extension='.wav')
dataset_train_non_aligned = SingleSequenceDataset(path_train_data_per, data_train_per, phone_labels)
data_loader_train = torch.utils.data.DataLoader(dataset_train_non_aligned, batch_size=BATCH_SIZE,
                                                shuffle=True)

data_val_per, _ = findAllSeqs(path_val_data_per, extension='.wav')
dataset_val_non_aligned = SingleSequenceDataset(path_val_data_per, data_val_per, phone_labels)
data_loader_val = torch.utils.data.DataLoader(dataset_val_non_aligned, batch_size=BATCH_SIZE,
                                              shuffle=True)

4it [00:00, 583.49it/s]

/content/CPC_audio
Saved cache file at /content/drive/My Drive/speech/data/train/_seqs_cache.txt



4it [00:00, 368.16it/s]

Loaded 143 sequences in 1.74 seconds
maxSizeSeq : 323433
maxSizePhone : 128
minSizePhone : 12
Total size dataset 0.39825538194444443 hours
Saved cache file at /content/drive/My Drive/speech/data/validation/_seqs_cache.txt


Loaded 143 sequences in 1.74 seconds
maxSizeSeq : 323433
maxSizePhone : 128
minSizePhone : 12
Total size dataset 0.39825538194444443 hours


### a- Training

Since the phonemes are not aligned, there is no simple direct way to get the classification acuracy of a model. Write and test the three functions ```train_one_epoch_ctc```, ```validation_step_ctc``` and ```run_ctc``` as before but without considering the average acuracy of the model. 

In [51]:
from cpc.feature_loader import loadModel

checkpoint_path = 'checkpoint_data/checkpoint_30.pt'
cpc_model, HIDDEN_CONTEXT_MODEL, HIDDEN_ENCODER_MODEL = loadModel([checkpoint_path])
cpc_model = cpc_model.cuda()
phone_classifier = PhoneClassifier(HIDDEN_CONTEXT_MODEL, N_PHONES).to(device)

Loading checkpoint checkpoint_data/checkpoint_30.pt
Loading the state dict at checkpoint_data/checkpoint_30.pt


In [52]:
parameters = list(phone_classifier.parameters()) + list(cpc_model.parameters())
LEARNING_RATE = 2e-4
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)

optimizer_frozen = torch.optim.Adam(list(phone_classifier.parameters()), lr=LEARNING_RATE)

In [53]:
import torch.nn.functional as F

def train_one_epoch_ctc(cpc_model, 
                        phone_classifier, 
                        loss_criterion, 
                        data_loader, 
                        optimizer):
  
  cpc_model.train()
  loss_criterion.train()

  avg_loss = 0
  avg_accuracy = 0
  n_items = 0
  for step, full_data in enumerate(data_loader):

    x, x_len, y, y_len = full_data

    x_batch_len = x.shape[-1]
    x, y = x.to(device), y.to(device)

    bs=x.size(0)
    optimizer.zero_grad()
    context_out, enc_out, _ = cpc_model(x.to(device),y.to(device))
  
    scores = phone_classifier(context_out)
    scores = scores.permute(1,0,2)
    scores = F.log_softmax(scores,2)
    yhat_len = torch.tensor([int(scores.shape[0]*x_len[i]/x_batch_len) for i in range(scores.shape[1])]) # this is an approximation, should be good enough

    loss = loss_criterion(scores,y.to(device),yhat_len,y_len)
    loss.backward()
    optimizer.step()
    avg_loss+=loss.item()*bs
    n_items+=bs
  avg_loss/=n_items
  return avg_loss

def validation_step(cpc_model, 
                    phone_classifier, 
                    loss_criterion, 
                    data_loader):

  cpc_model.eval()
  phone_classifier.eval()
  avg_loss = 0
  avg_accuracy = 0
  n_items = 0
  with torch.no_grad():
    for step, full_data in enumerate(data_loader):

      x, x_len, y, y_len = full_data

      x_batch_len = x.shape[-1]
      x, y = x.to(device), y.to(device)

      bs=x.size(0)
      context_out, enc_out, _ = cpc_model(x.to(device),y.to(device))
    
      scores = phone_classifier(context_out)
      scores = scores.permute(1,0,2)
      scores = F.log_softmax(scores,2)
      yhat_len = torch.tensor([int(scores.shape[0]*x_len[i]/x_batch_len) for i in range(scores.shape[1])]) # this is an approximation, should be good enough

      loss = loss_criterion(scores,y.to(device),yhat_len,y_len)
      avg_loss+=loss.item()*bs
      n_items+=bs
  avg_loss/=n_items

  return avg_loss

def run_ctc(cpc_model, 
            phone_classifier, 
            loss_criterion, 
            data_loader_train, 
            data_loader_val, 
            optimizer,
            n_epoch):
  for epoch in range(n_epoch):

    print(f"Running epoch {epoch + 1} / {n_epoch}")
    loss_train = train_one_epoch_ctc(cpc_model, phone_classifier, loss_criterion, data_loader_train, optimizer)
    print("-------------------")
    print(f"Training dataset :")
    print(f"Average loss : {loss_train}.")

    print("-------------------")
    print("Validation dataset")
    loss_val = validation_step(cpc_model, phone_classifier, loss_criterion, data_loader_val)
    print(f"Average loss : {loss_val}")
    print("-------------------")
    print()

In [54]:
run_ctc(cpc_model,phone_classifier,loss_ctc,data_loader_train,data_loader_val,optimizer_frozen,n_epoch=10)

Running epoch 1 / 10
-------------------
Training dataset :
Average loss : 72.6621567632111.
-------------------
Validation dataset
Average loss : 71.02616280569157
-------------------

Running epoch 2 / 10


KeyboardInterrupt: ignored

### b- Evaluation: the Phone Error Rate (PER)

In order to compute the similarity between two sequences, we can use the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance). This distance estimates the minimum number of insertion, deletion and addition to move from one sequence to another. If we normalize this distance by the number of characters in the reference sequence we get the Phone Error Rate (PER).

This value can be interpreted as :
\\[  PER = \frac{S + D + I}{N} \\]

Where:


*   N is the number of characters in the reference
*   S is the number of substitutiion
*   I in the number of insertion
*   D in the number of deletion

For the best possible alignment of the two sequences.



In [55]:
import numpy as np

def get_PER_sequence(ref_seq, target_seq):

  # re = g.split()
  # h = h.split()
  n = len(ref_seq)
  m = len(target_seq)

  D = np.zeros((n+1,m+1))
  for i in range(1,n+1):
    D[i,0] = D[i-1,0]+1
  for j in range(1,m+1):
    D[0,j] = D[0,j-1]+1
  
  ### TODO compute the alignment

  for i in range(1,n+1):
    for j in range(1,m+1):
      D[i,j] = min(
          D[i-1,j]+1,
          D[i-1,j-1]+1,
          D[i,j-1]+1,
          D[i-1,j-1]+ 0 if ref_seq[i-1]==target_seq[j-1] else float("inf")
      )
  return D[n,m]/len(ref_seq)
  

  #return PER

You can test your function below:

In [56]:
ref_seq = [0, 1, 1, 2, 0, 2, 2]
pred_seq = [1, 1, 2, 2, 0, 0]

expected_PER = 4. / 7.
print(get_PER_sequence(ref_seq, pred_seq) == expected_PER)

True


## c- Evaluating the PER of your model on the test dataset

Evaluate the PER on the validation dataset. Please notice that you should usually use a separate dataset, called the dev dataset, to perform this operation. However for the sake of simplicity we will work with validation data in this exercise.

In [57]:
import progressbar
from multiprocessing import Pool

def cut_data(seq, sizeSeq):
    maxSeq = sizeSeq.max()
    return seq[:, :maxSeq]


def prepare_data(data):
    seq, sizeSeq, phone, sizePhone = data
    seq = seq.cuda()
    phone = phone.cuda()
    sizeSeq = sizeSeq.cuda().view(-1)
    sizePhone = sizePhone.cuda().view(-1)

    seq = cut_data(seq.permute(0, 2, 1), sizeSeq).permute(0, 2, 1)
    return seq, sizeSeq, phone, sizePhone


def get_per(test_dataloader,
            cpc_model,
            phone_classifier):

  downsampling_factor = 160
  cpc_model.eval()
  phone_classifier.eval()

  avgPER = 0
  nItems = 0 

  print("Starting the PER computation through beam search")
  bar = progressbar.ProgressBar(maxval=len(test_dataloader))
  bar.start()

  for index, data in enumerate(test_dataloader):

    bar.update(index)

    with torch.no_grad():
      
        seq, sizeSeq, phone, sizePhone = prepare_data(data)
        c_feature, _, _ = cpc_model(seq.to(device),phone.to(device))
        sizeSeq = sizeSeq / downsampling_factor
        predictions = torch.nn.functional.softmax(
        phone_classifier(c_feature), dim=2).cpu()
        phone = phone.cpu()
        sizeSeq = sizeSeq.cpu()
        sizePhone = sizePhone.cpu()

        bs = c_feature.size(0)
        data_per = [(predictions[b].argmax(1),  phone[b]) for b in range(bs)]
        # data_per = [(predictions[b], sizeSeq[b], phone[b], sizePhone[b],
        #               "criterion.module.BLANK_LABEL") for b in range(bs)]

        with Pool(bs) as p:
            poolData = p.starmap(get_PER_sequence, data_per)
        avgPER += sum([x for x in poolData])
        nItems += len(poolData)

  bar.finish()

  avgPER /= nItems

  print(f"Average PER {avgPER}")
  return avgPER


In [58]:
get_per(data_loader_val,cpc_model,phone_classifier)

N/A% (0 of 18) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Starting the PER computation through beam search


 11% (2 of 18) |##                       | Elapsed Time: 0:00:29 ETA:   0:03:06Process ForkPoolWorker-478:
Process ForkPoolWorker-482:
Process ForkPoolWorker-484:
Process ForkPoolWorker-483:
Process ForkPoolWorker-480:
Traceback (most recent call last):
Process ForkPoolWorker-481:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-477:


KeyboardInterrupt: ignored

## Exercice 3 : Character error rate (CER) 

The Character Error Rate (CER) is an evaluation metric similar to the PER but with characters insterad of phonemes. Using the following data, run the functions you defined previously to estimate the CER of your model after fine-tuning.

In [62]:
# Load a dataset labelled with the letters of each sequence.
%cd /content/CPC_audio
from cpc.eval.common_voices_eval import SingleSequenceDataset, parseSeqLabels, findAllSeqs
 
path_train_data_cer = '/content/drive/My Drive/speech/data/train'
path_val_data_cer = '/content/drive/My Drive/speech/data/validation'
path_test_data_cer = '/content/drive/My Drive/speech/data/test'
path_letter_data_cer = '/content/drive/My Drive/speech/all_sessions.txt'

BATCH_SIZE=8

letters_labels, N_LETTERS = parseSeqLabels(path_letter_data_cer)
data_train_cer, _ = findAllSeqs(path_train_data_cer, extension='.wav')
dataset_train_non_aligned = SingleSequenceDataset(path_train_data_cer, data_train_cer, letters_labels)


data_val_cer, _ = findAllSeqs(path_val_data_cer, extension='.wav')
dataset_val_non_aligned = SingleSequenceDataset(path_val_data_cer, data_val_cer, letters_labels)


# The data loader will generate a tuple of tensors data, labels for each batch
# data : size N x T1 x 1 : the audio sequence
# label : size N x T2 the sequence of letters corresponding to the audio data
# IMPORTANT NOTE: just like the PER the CER is computed with non-aligned phone data.
data_loader_train_letters = torch.utils.data.DataLoader(dataset_train_non_aligned, batch_size=BATCH_SIZE,
                                                shuffle=True)
data_loader_val_letters = torch.utils.data.DataLoader(dataset_val_non_aligned, batch_size=BATCH_SIZE,
                                              shuffle=True)

4it [00:00, 570.29it/s]

/content/CPC_audio
Saved cache file at /content/drive/My Drive/speech/data/train/_seqs_cache.txt



4it [00:00, 422.71it/s]

Loaded 143 sequences in 1.85 seconds
maxSizeSeq : 323433
maxSizePhone : 128
minSizePhone : 12
Total size dataset 0.39825538194444443 hours
Saved cache file at /content/drive/My Drive/speech/data/validation/_seqs_cache.txt


Loaded 143 sequences in 1.84 seconds
maxSizeSeq : 323433
maxSizePhone : 128
minSizePhone : 12
Total size dataset 0.39825538194444443 hours


In [63]:
from cpc.feature_loader import loadModel

checkpoint_path = 'checkpoint_data/checkpoint_30.pt'
cpc_model, HIDDEN_CONTEXT_MODEL, HIDDEN_ENCODER_MODEL = loadModel([checkpoint_path])
cpc_model = cpc_model.cuda()
character_classifier = PhoneClassifier(HIDDEN_CONTEXT_MODEL, N_LETTERS).to(device)

Loading checkpoint checkpoint_data/checkpoint_30.pt
Loading the state dict at checkpoint_data/checkpoint_30.pt


In [64]:
parameters = list(character_classifier.parameters()) + list(cpc_model.parameters())
LEARNING_RATE = 2e-4
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)

optimizer_frozen = torch.optim.Adam(list(character_classifier.parameters()), lr=LEARNING_RATE)

In [65]:
loss_ctc = torch.nn.CTCLoss()

In [66]:
run_ctc(cpc_model,character_classifier,loss_ctc,data_loader_train_letters,data_loader_val_letters,optimizer_frozen,n_epoch=10)

Running epoch 1 / 10
-------------------
Training dataset :
Average loss : 74.07048206597986.
-------------------
Validation dataset
Average loss : 72.3986094300176
-------------------

Running epoch 2 / 10
-------------------
Training dataset :
Average loss : 70.69946375027509.
-------------------
Validation dataset
Average loss : 68.68825015215806
-------------------

Running epoch 3 / 10
-------------------
Training dataset :
Average loss : 66.85569730946716.
-------------------
Validation dataset
Average loss : 64.64670987196372
-------------------

Running epoch 4 / 10
-------------------
Training dataset :
Average loss : 62.66919170970648.
-------------------
Validation dataset
Average loss : 60.55429555328799
-------------------

Running epoch 5 / 10
-------------------
Training dataset :
Average loss : 58.553509779379404.
-------------------
Validation dataset
Average loss : 56.37082059618453
-------------------

Running epoch 6 / 10
-------------------
Training dataset :
Avera

In [67]:
get_per(data_loader_val_letters,cpc_model,character_classifier)

N/A% (0 of 18) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Starting the PER computation through beam search


100% (18 of 18) |########################| Elapsed Time: 0:04:26 Time:  0:04:26


Average PER 0.9346970913183367


0.9346970913183367